# 数据收集

## 融资融券标的

In [1]:
import os
os.chdir(r'D:\paper\rowdata')

import pandas as pd
import numpy as np

In [2]:
rzrq = pd.read_csv('rzrq.csv', parse_dates=[2, 3], infer_datetime_format=True,
                   dtype={'标的证券代码_UdlSecuCd':str}, encoding='utf-8').iloc[:, 0:5]

rzrq = rzrq.rename(columns = {"标的证券代码_UdlSecuCd": "Code", "标的类别_UdlCtgy": "Type",
                              "入选日期_InDt": "Time_in", "剔除日期_OutDt": "Time_out",
                              "标的标志_UdlFlg": "Biaodi", "变更原因_ChgRea": "Reason"})

---

去重

In [3]:
rzrq = rzrq.drop_duplicates(["Code", "Time_in", "Time_out", "Biaodi"])

---

In [4]:
rzrq_2013 = pd.read_csv('rzrq_2013.csv', parse_dates=[1], infer_datetime_format=True,
                       dtype={'Stockcode': str}).iloc[:, [1,2,4,5]]
rzrq_2017 = pd.read_csv('rzrq_2017.csv', parse_dates=[1], infer_datetime_format=True,
                       dtype={'Stockcode': str}).iloc[:, [1,2,4,5]]
rzrq_2021_1 = pd.read_csv('rzrq_2021_1.csv', parse_dates=[1], infer_datetime_format=True,
                       dtype={'Stockcode': str}).iloc[:, [1,2,4,5]]
rzrq_2021_2 = pd.read_csv('rzrq_2021_2.csv', parse_dates=[1], infer_datetime_format=True,
                       dtype={'Stockcode': str}).iloc[:, [1,2,4,5]]
rzrq_jy = pd.concat([rzrq_2013, rzrq_2017, rzrq_2021_1, rzrq_2021_2])
rzrq_jy

,Mtdate,Stockcode,Financeavailable,Shortavailable
0,2010-03-29,000002,N,N
1,2010-03-30,000002,N,N
2,2010-03-31,000002,Y,Y
3,2010-04-01,000002,Y,Y
4,2010-04-02,000002,Y,Y
...,...,...,...,...
69984,2021-12-27,603993,Y,Y
69985,2021-12-28,603993,Y,Y
69986,2021-12-29,603993,Y,Y
69987,2021-12-30,603993,Y,Y


1,987,905条数据，excel早卡死了，还得是python.

In [48]:
ff = rzrq_jy[rzrq_jy["Stockcode"]=='000002']
str(ff[ff["Mtdate"]>='2021-10']["Mtdate"].iloc[-1])[:10]

'2021-12-31'

---

接下来挑出真正的剔除数据  
上面来自锐思的数据中，有些股票曾今被剔除过，然而并非是“真正”被剔除出融资融券标的  
其中有些股票是触发了融资融券交易的暂停机制，这些数据也被当作了剔除


要识别出这样的假“剔除”，笔者的思路如下：  
我们先挑选出所有的“被剔除”的记录，而后  
1. 先检查在2021-12-31其是否还是融资融券标的，如果是的话则视其为为“假”剔除数据，为稳妥起见，再查看2021-11-11、2021-10-11其是否是标的  
2. 还有一种可能，证券曾经被剔除过标的名单，但在之后的扩容中，其又被纳入了标的，这种情况，笔者认为，不妨认为其也是一种暂停
3. 对于“真”剔除，我们返回其被剔除的时间

得到被“剔除”过的证券的名单：

In [5]:
get_out_df = rzrq[rzrq["Time_out"].notnull()]
get_out = set(get_out_df['Code'])

In [6]:
real_get_out = []

for item in get_out:
    q = rzrq_jy[rzrq_jy["Stockcode"]==item]
    da = str(q["Mtdate"].iloc[-1])[:10]

    if da == "2021-12-31":
        continue
    else:
        bi = q["Financeavailable"].iloc[-1]
        real_get_out.append((item, da))

In [7]:
real_get_out

[('002426', '2020-04-29'),
 ('002650', '2020-06-12'),
 ('600470', '2020-04-30'),
 ('002501', '2019-01-11'),
 ('600179', '2016-03-10'),
 ('600381', '2013-03-05'),
 ('600226', '2020-06-30'),
 ('000788', '2017-07-07'),
 ('600198', '2021-04-29'),
 ('600403', '2021-04-27'),
 ('002524', '2017-07-07'),
 ('600225', '2020-04-23'),
 ('600186', '2019-04-26'),
 ('002321', '2021-04-29'),
 ('600815', '2017-04-28'),
 ('002716', '2019-10-08'),
 ('600311', '2015-11-30'),
 ('600228', '2014-04-30'),
 ('600526', '2019-04-19'),
 ('000410', '2017-03-17'),
 ('600518', '2019-05-20'),
 ('601258', '2019-09-06'),
 ('600112', '2017-04-26'),
 ('000626', '2018-10-12'),
 ('000897', '2018-10-12'),
 ('600515', '2021-02-18'),
 ('002432', '2014-09-19'),
 ('002122', '2013-01-30'),
 ('002358', '2021-04-29'),
 ('600119', '2019-04-30'),
 ('600193', '2018-05-02'),
 ('002490', '2017-03-17'),
 ('600078', '2021-04-30'),
 ('002571', '2017-04-24'),
 ('600301', '2013-05-02'),
 ('603188', '2018-08-13'),
 ('600807', '2018-05-02'),
 

草，终于解决了。。。

In [8]:
code_ls = []
with open("Code.txt",'r') as code:
    for line in code:
        code_ls.append(line[:-1])

In [9]:
df = pd.DataFrame({"Code": code_ls})

num_code = len(code_ls)

df["Treated"] = [0 for i in range(num_code)]
df["Tc_t"] = [0 for i in range(num_code)]
df["Time"] = [0 for i in range(num_code)]
df["Time_a"] =[0 for i in range(num_code)]

In [10]:
treated = set(rzrq["Code"])
treated_ls = [1 if i in treated else 0 for i in df["Code"]]
df["Treated"] = treated_ls

real_get_out_code = [i[0] for i in real_get_out]
real_get_out_dict = {i[0]:i[1] for i in real_get_out}
tc_t = [1 if  i in real_get_out_code else 0 for i in code_ls]
df["Tc_t"] = tc_t

Time_a = [real_get_out_dict[i] if  i in real_get_out_code else np.nan for i in code_ls]
df["Time_a"] = Time_a

Time = [str(rzrq[rzrq["Code"]==i]["Time_in"].iloc[0])[:10] if  i in treated else np.nan for i in code_ls]
df["Time"] = Time

df

,Code,Treated,Tc_t,Time,Time_a
0,000002,1,0,2010-03-29,NaN
1,000004,0,0,NaN,NaN
2,000005,0,0,NaN,NaN
3,000006,1,0,2013-01-31,NaN
4,000007,0,0,NaN,NaN
...,...,...,...,...,...
2971,605580,0,0,NaN,NaN
2972,605588,0,0,NaN,NaN
2973,605589,0,0,NaN,NaN
2974,605598,0,0,NaN,NaN


舒服了，草  
将得到的数据保存：

In [11]:
df.to_csv('rzrq_clean.csv', index=False)